This Jupyter notebook file will collect training data and format it into a way where it can be processed by the stockpred model.

In [1]:
from stockinfo import extract_tickers_from_directory

directory = "../../data/earning_calls"
tickers_data = extract_tickers_from_directory(directory)
print(tickers_data)


[{'ticker': 'GPN', 'start_date': '20160728'}, {'ticker': 'K', 'start_date': '20170209'}, {'ticker': 'DDD', 'start_date': '20150506'}, {'ticker': 'DLR', 'start_date': '20160728'}, {'ticker': 'CMS', 'start_date': '20151029'}, {'ticker': 'AVY', 'start_date': '20170426'}, {'ticker': 'CAH', 'start_date': '20170501'}, {'ticker': 'SPPI', 'start_date': '20160809'}, {'ticker': 'TWI', 'start_date': '20160804'}, {'ticker': 'NATI', 'start_date': '20150728'}, {'ticker': 'CNK', 'start_date': '20160510'}, {'ticker': 'MYGN', 'start_date': '20150505'}, {'ticker': 'JCI', 'start_date': '20170427'}, {'ticker': 'WY', 'start_date': '20150731'}, {'ticker': 'FB', 'start_date': '20171101'}, {'ticker': 'PYPL', 'start_date': '20160127'}, {'ticker': 'CUB', 'start_date': '20160502'}, {'ticker': 'D', 'start_date': '20170802'}, {'ticker': 'CVLT', 'start_date': '20161025'}, {'ticker': 'PRAA', 'start_date': '20150810'}, {'ticker': 'PEI', 'start_date': '20170224'}, {'ticker': 'COLB', 'start_date': '20180125'}, {'ticker

In [2]:
import random
import pandas as pd
from datetime import datetime, timedelta

def add_prediction_date(start_date, prediction_days):
    """
    Converts a start_date string to datetime and adds prediction_days.

    Args:
        start_date (str): Start date in the format 'YYYYMMDD'.
        prediction_days (int): Number of days to add for prediction.

    Returns:
        str: The resulting date in the format 'YYYYMMDD'.
    """
    start_date_dt = datetime.strptime(start_date, '%Y%m%d')
    start_date_str = start_date_dt.strftime('%Y-%m-%d')
    prediction_date_dt = start_date_dt + timedelta(days=prediction_days)
    prediction_date_str = prediction_date_dt.strftime('%Y-%m-%d')

    return start_date_str, prediction_date_str


data = tickers_data
df = pd.DataFrame(data)

df['prediction_date'] = df['start_date'].apply(
    # TODO: What should we make this?
    lambda x: 90
)

for index, row in df.iterrows():
    prediction_days = row['prediction_date']
    start_date = row['start_date']
    start_date_str, prediction_date_str = add_prediction_date(start_date, prediction_days)
    df.at[index, 'start_date'] = start_date_str
    df.at[index, 'prediction_date'] = prediction_date_str

print(df)


     ticker  start_date prediction_date
0       GPN  2016-07-28      2016-10-26
1         K  2017-02-09      2017-05-10
2       DDD  2015-05-06      2015-08-04
3       DLR  2016-07-28      2016-10-26
4       CMS  2015-10-29      2016-01-27
...     ...         ...             ...
3438    GPS  2016-05-19      2016-08-17
3439   WTFC  2016-10-18      2017-01-16
3440   MSCI  2017-05-04      2017-08-02
3441    JWN  2015-11-12      2016-02-10
3442   MPWR  2017-02-09      2017-05-10

[3443 rows x 3 columns]


/var/folders/15/8q1pvppj40b6gcq695yybpcc0000gn/T/ipykernel_22195/455664704.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2016-10-26' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'prediction_date'] = prediction_date_str


In [3]:
# Add earnings call sentiment scores
sentiment_df = pd.read_csv('sentiment_results.csv')

df['sentiment_score'] = sentiment_df['sentiment_intensity']

In [4]:
df

,ticker,start_date,prediction_date,sentiment_score
0,GPN,2016-07-28,2016-10-26,0.219453
1,K,2017-02-09,2017-05-10,0.138828
2,DDD,2015-05-06,2015-08-04,0.221033
3,DLR,2016-07-28,2016-10-26,0.276153
4,CMS,2015-10-29,2016-01-27,0.123286
...,...,...,...,...
3438,GPS,2016-05-19,2016-08-17,0.341247
3439,WTFC,2016-10-18,2017-01-16,0.054652
3440,MSCI,2017-05-04,2017-08-02,0.912142
3441,JWN,2015-11-12,2016-02-10,0.503056


(For finding correlation demo)

In [ ]:
df = df.sample(n=50)

In [5]:
from stockinfo import collect_all_features

dataset = {}
num_stocks = 500

# For each row in df, collect all features during that time period
for index, row in df.iterrows():
    try:
        ticker = row['ticker']
        start_date = row['start_date']
        prediction_date = row['prediction_date']

        features = collect_all_features(ticker, start_date, prediction_date, None, None, None)
        features['Earnings_Call_Sentiment'] = row['sentiment_score']
        features.dropna(inplace=True)
        dataset[ticker] = features

        if len(dataset.keys()) >= num_stocks:
            break
    except:
        continue
dataset

$SPPI: possibly delisted; no timezone found
$NATI: possibly delisted; no timezone found
$FB: possibly delisted; no timezone found
$CUB: possibly delisted; no price data found  (1d 2015-08-26 -> 2016-07-31) (Yahoo error = "Data doesn't exist for startDate = 1440561600, endDate = 1469937600")
$PEI: possibly delisted; no timezone found
$INT: possibly delisted; no timezone found
$PES: possibly delisted; no timezone found
$IVC: possibly delisted; no timezone found
$CPE: possibly delisted; no timezone found
$TPRE: possibly delisted; no timezone found
$WWE: possibly delisted; no timezone found
$PLT: possibly delisted; no timezone found
$ADS: possibly delisted; no timezone found
$INTL: possibly delisted; no price data found  (1d 2015-08-29 -> 2016-08-03) (Yahoo error = "Data doesn't exist for startDate = 1440820800, endDate = 1470196800")
$RUTH: possibly delisted; no timezone found
$BBBY: possibly delisted; no timezone found
$NATI: possibly delisted; no timezone found
$ADS: possibly delisted; 

{'GPN':                                 Open       High        Low      Close  \
 Date                                                                    
 2016-07-28 00:00:00-04:00  75.013740  75.716246  71.597468  75.244705   
 2016-07-29 00:00:00-04:00  75.119585  75.167704  71.818788  71.847664   
 2016-08-01 00:00:00-04:00  71.866927  72.271104  70.558155  70.933464   
 2016-08-02 00:00:00-04:00  70.558145  70.683245  68.508381  68.806702   
 2016-08-03 00:00:00-04:00  68.671983  70.538908  68.113829  70.404182   
 ...                              ...        ...        ...        ...   
 2016-10-19 00:00:00-04:00  71.462650  71.722511  71.366406  71.578148   
 2016-10-20 00:00:00-04:00  71.472300  71.645543  70.750455  70.827446   
 2016-10-21 00:00:00-04:00  70.288452  70.375077  69.807222  70.144081   
 2016-10-24 00:00:00-04:00  70.692676  71.472274  70.654184  71.279778   
 2016-10-25 00:00:00-04:00  71.279784  71.279784  70.047836  70.519440   
 
                             

(Demo: Finding correlation)

In [ ]:
from stockinfo import calculate_correlation

corr_df, corr_score = calculate_correlation(dataset)
corr_df, corr_score


In [ ]:
import numpy as np
from typing import Tuple
def prepare_sequence_data(data: np.ndarray, 
                            look_back: int, 
                            look_forward: int, 
                            target_idx: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Prepare sequence data from raw time series data.
        
        Args:
            data: Raw time series data of shape (n_timestamps, n_features)
            look_back: Number of time steps to look back
            look_forward: Number of time steps to predict ahead
            target_idx: Index of the target feature
            
        Returns:
            Tuple of (X, y) where:
                X: Sequences of shape (n_samples, look_back, n_features)
                y: Target values of shape (n_samples,)
        """
        if len(data.shape) != 2:
            raise ValueError(f"Expected 2D input array, got shape {data.shape}")
            
        if target_idx >= data.shape[1]:
            raise ValueError(f"target_idx {target_idx} must be less than number of features {data.shape[1]}")
        
        X, y = [], []
        for i in range(len(data) - look_back - look_forward):
            X.append(data[i:(i + look_back)])
            y.append(data[i + look_back + look_forward - 1, target_idx])
        
        return np.array(X), np.array(y)

In [ ]:
processed_data = {}
features = [column for column in dataset['GPN'].columns if column != 'Dividends' and column != 'Stock Splits']
target_column = 'Close'
target_idx = 3
look_back = 10
look_forward = 5

print(features)
for ticker, data in dataset.items():
    X, y = prepare_sequence_data(data[features].values, look_back, look_forward, target_idx)
    # check that each has three dimension
    if len(X.shape) != 3:
        print(f"Skipping {ticker} (unexpected X shape: {X.shape})")
        continue

    processed_data[ticker] = {'X': X, 'y': y}

# Use last ticker as test data
test_data = processed_data.popitem()
X_test = test_data[1]['X']
y_test = test_data[1]['y']

np.save('../../models/X_test.npy', X_test)
np.save('../../models/y_test.npy', y_test)

X_combined = np.vstack([seq['X'] for seq in processed_data.values()])
y_combined = np.concatenate([seq['y'] for seq in processed_data.values()])

In [ ]:
X_combined.shape, y_combined.shape

In [ ]:
np.save('../../models/X_combined.npy', X_combined)
np.save('../../models/y_combined.npy', y_combined)

Get the rest of the features:

In [ ]:
from stockinfo import collect_all_features 

print(len(df))
sampled_df = df.sample(n=20)

def add_features_to_df(df, collect_features_func):
    aggregated_features = [] 

    for _, row in df.iterrows():
        ticker = row['ticker']
        start_date = row['start_date']
        prediction_date = row['prediction_date']

        try:
            features_df = collect_features_func(
                ticker=ticker,
                start_date=start_date,
                end_date=prediction_date,
                reddit_data_func=None,
                sentiment_func=None,
                time_series_func=None
            )

            aggregated_row = {
                'ticker': ticker,
                'start_date': start_date,
                'prediction_date': prediction_date,
                'Open_mean': features_df['Open'].mean(),
                'Close_mean': features_df['Close'].mean(),
                'High_max': features_df['High'].max(),
                'Low_min': features_df['Low'].min(),
                'Volume_sum': features_df['Volume'].sum(),
                'Volatility_mean': features_df['Volatility'].mean(),
                'RSI_mean': features_df['RSI'].mean(),
                'MACD_mean': features_df['MACD'].mean(),
                'Signal_Line_mean': features_df['Signal_Line'].mean(),
                'BB_Upper_last': features_df['BB_Upper'].iloc[-1],
                'BB_Lower_last': features_df['BB_Lower'].iloc[-1],
                'Momentum_last': features_df['Momentum'].iloc[-1],
                'MA_7_last': features_df['MA_7'].iloc[-1] if 'MA_7' in features_df else None,
                'MA_14_last': features_df['MA_14'].iloc[-1] if 'MA_14' in features_df else None,
                'MA_50_last': features_df['MA_50'].iloc[-1] if 'MA_50' in features_df else None,
                # Mock Reddit features
                #'Reddit_Mentions': features_df['Reddit_Mentions'].sum() if 'Reddit_Mentions' in features_df else None,
                #'Reddit_Sentiment': features_df['Reddit_Sentiment'].iloc[-1] if 'Reddit_Sentiment' in features_df else None,
                # Mock sentiment features
                #'Earnings_Sentiment': features_df['Earnings_Sentiment'].iloc[-1] if 'Earnings_Sentiment' in features_df else None,
                #'Sentiment_Intensity': features_df['Sentiment_Intensity'].iloc[-1] if 'Sentiment_Intensity' in features_df else None,
                # Mock time-series prediction
                #'Predicted_Price': features_df['Predicted_Price'].iloc[-1] if 'Predicted_Price' in features_df else None
            }

            aggregated_features.append(aggregated_row)

        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            continue

    features_df = pd.DataFrame(aggregated_features)
    return features_df

features_df = add_features_to_df(sampled_df, collect_all_features)
print(features_df)



Now, compute the training labels: 

In [ ]:
import yfinance as yf

def get_labels(df):

    labels = []

    for _, row in df.iterrows():
        ticker = row['ticker']
        start_date = row['start_date']
        prediction_date = row['prediction_date']

        try:
            stock = yf.Ticker(ticker)
            stock_data = stock.history(start=start_date, end=prediction_date)

            if not stock_data.empty:
                prediction_price = stock_data.loc[stock_data.index[-1]]['Close']
            else:
                prediction_price = None

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            prediction_price = None

        labels.append(prediction_price)

    df['label'] = labels
    return df

features_df = get_labels(features_df)
print(features_df[["ticker","start_date","prediction_date","label"]])


The code below will split these inputs into training and testing datasets:

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(features_df, test_size=0.2)

print(train_df)
print(test_df)

EXAMPLE: get reddit mentions specifically for Boeing

In [ ]:
from stockinfo import collect_reddit_data
import sys

# get the query

ticker = "BA"
stock = yf.Ticker(ticker).get_info()
longName = stock["longName"]

name = ticker
if len(longName.split(" ")[0]) > 2:
    if longName.split(" ")[0] == "The":
        name = longName.split(" ")[1]
    else:
        name = longName.split(" ")[0]
query = f'{ticker} OR \"{ticker} stock\" OR \"{longName}\" OR {name} OR \"{name} stock\"'
print(query)

# check if reddit file already exists in folder structure-- if not, collect reddit data

reddit_file = f"../../data/reddit_data/{ticker}/reddit_comments.csv"

if os.path.exists(reddit_file):
    print(f"Reddit data file already exists for {ticker}. Loading data...")
    reddit_data = pd.read_csv(reddit_file)
else:
    print(f"Reddit data file does not exist for {ticker}. Must run data collection script?")

# run collect_reddit_data() on the ticker

res = collect_reddit_data(ticker, query)
print(res)

# get the earnings call related to the ticker

direc = "../../data/earning_calls"

for root, _, files in os.walk(direc):
    if root.endswith("_"+ticker):
        direc = root
        break

# get the sentiment from the earnings call

module_path = os.path.abspath(os.path.join('../..', 'models'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sentiment import finbert_handler

sent = finbert_handler(data_input=direc) # this might be wrong... need to test. From what I've seen Boeing should be negative
# Before fully implementing this... need to meet with Ben and figure out how to eliminate filler and cleaning in general

print(sent)

TODO: change collect_data.ipynb to collect reddit dataset for all 3000 earnings calls. Make rule where if less than 10 mentions, set sentiment to Neutral.

## Creating Time Series Dataset

First, we obtain all the tickers and dates for the earning calls we have data for.

In [ ]:
directory = "../../data/earning_calls"
tickers_data = extract_tickers_from_directory(directory)
print(tickers_data)
